In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "/content/ntu_encoder/"
!mkdir "/content/ntu_encoder/train"
!mkdir "/content/ntu_encoder/test"

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/ntu-rgb-120.zip" "/content/"
!ls -l "/content/"
import zipfile
zip_ref = zipfile.ZipFile("/content/ntu-rgb-120.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()
!rm "/content/ntu-rgb-120.zip"

total 10915252
drwx------ 6 root root        4096 Mar  4 18:39 drive
drwxr-xr-x 4 root root        4096 Mar  4 18:39 ntu_encoder
-rw------- 1 root root 11177194741 Mar  4 18:41 ntu-rgb-120.zip
drwxr-xr-x 1 root root        4096 Feb 18 14:33 sample_data


In [ ]:
from threading import Thread
from queue import Queue
from urllib.request import urlopen, Request
import json
from os import listdir
from os import makedirs
from random import seed
from random import random
import numpy as np
import cv2
import time

max_frames = 40

In [ ]:
id_classes = ['A001','A002','A003','A004','A005','A006','A007','A008','A009',
    'A010','A011','A012','A013','A014','A015','A016','A017','A018','A019',
    'A020','A021','A022','A023','A024','A025','A026','A027','A028','A029',
    'A030','A031','A032','A033','A034','A035','A036','A037','A038','A039',
    'A040','A041','A042','A043','A044','A045','A046','A047','A048','A049',
    'A050','A051','A052','A053','A054','A055','A056','A057','A058','A059',
    'A060','A061','A062','A063','A064','A065','A066','A067','A068','A069',
    'A070','A071','A072','A073','A074','A075','A076','A077','A078','A079',
    'A080','A081','A082','A083','A084','A085','A086','A087','A088','A089',
    'A090','A091','A092','A093','A094','A095','A096','A097','A098','A099',
    'A100','A101','A102','A103','A104','A105','A106','A107','A108','A109',
    'A110','A111','A112','A113','A114','A115','A116','A117','A118','A119','A120']
classes = ['drink water','eat meal','brush teeth','brush hair','drop','pick up','throw','sit down',
           'stand up','clapping','reading','writing','tear up paper','put on jacket','take off jacket',
           'put on a shoe','take off a shoe','put on glasses','take off glasses','put on a hat cap','take off a hat cap',
           'cheer up','hand waving','kicking something','reach into pocket','hopping','jump up','phone call',
           'play with phone  tablet','type on a keyboard','point to something','taking a selfie','check time',
           'rub two hands','nod head bow','shake head','wipe face','salute','put palms together','cross hands in front',
           'sneeze cough','staggering','falling down','headache','chest pain','back pain','neck pain','nausea vomiting',
           'fan self','punch slap','kicking','pushing','pat on back','point finger','hugging','giving object','touch pocket',
           'shaking hands','walking towards','walking apart','put on headphone','take off headphone',
           'shoot at basket','bouce ball','tennis bat swing','juggle bat tennis ball','hush','flick hair',
           'thumb up','thumb down','make OK sign','make victory sign','staple book','couting money',
           'cutting nails','cutting paper','snap fingers','open bottle','sniff smell','squat down','toss a coin',
           'fold paper','bull up paper','play magic cube','apply cream on face','apply cream on hand','put on bag',
           'take off bag','put object into bag','take object out of bag','open a box','move heavy objects',
           'shake fist','throw up cap hat','capitulate','cross arms','arm circles','arm swings',
           'run on the spots','butt kicks','cross toe touch','side kick','yawn','stretch one self','blow nose',
           'hit with object','wield knife','knock over','grab stuff','shoot with gun','step on foot','high-five',
           'cheers and drink','carry object','take a photo','follow','whisper','exchange things',
           'support somebody','rock-paper-scissors']

classes_alphab = ['apply cream on face','apply cream on hand','arm circles','arm swings','back pain','blow nose','bouce ball','brush hair',
           'brush teeth','bull up paper','butt kicks','capitulate','carry object','check time','cheer up',
           'cheers and drink','chest pain','clapping','couting money','cross arms','cross hands in front',
           'cross toe touch','cutting nails','cutting paper','drink water','drop','eat meal','exchange things',
           'falling down','fan self','flick hair','fold paper','follow',
           'giving object','grab stuff','hand waving','headache','high-five','hit with object','hopping',
           'hugging','hush','juggle bat tennis ball','jump up','kicking','kicking something','knock over','make OK sign',
           'make victory sign','move heavy objects','nausea vomiting','neck pain','nod head bow','open a box','open bottle','pat on back','phone call',
           'pick up','play magic cube','play with phone  tablet','point finger','point to something',
           'punch slap','pushing','put object into bag','put on a hat cap','put on a shoe','put on bag',
           'put on glasses','put on headphone','put on jacket','put palms together','reach into pocket','reading',
           'rock-paper-scissors','rub two hands','run on the spots','salute','shake fist','shake head','shaking hands',
           'shoot at basket','shoot with gun','side kick','sit down','snap fingers','sneeze cough',
           'sniff smell','squat down','staggering','stand up','staple book',
           'step on foot','stretch one self','support somebody','take a photo','take object out of bag','take off a hat cap',
           'take off a shoe','take off bag','take off glasses','take off headphone','take off jacket','taking a selfie','tear up paper',
           'tennis bat swing','throw','throw up cap hat','thumb down','thumb up','toss a coin','touch pocket',
           'type on a keyboard','walking apart','walking towards','whisper','wield knife','wipe face',
           'writing','yawn']
      
dataset_home = "/content/ntu_encoder/"
subdirs = ["train/","test/"]
for subdir in subdirs:
    for labdir in classes:
        newdir = dataset_home + subdir + labdir
        makedirs(newdir,exist_ok=True)

In [ ]:
def to_nassim(data, type='foot_to_foot'):
    data = [get_data(x,type) for x in data]
    image = transform_image_ludl(data)
    
    return image

def get_data(datait,type='no_order'):
    if type=='no_order':
        data = np.asarray(datait)
        data = data.reshape((25,3))
    else:
        spine_base = datait[0]
        spine_mid = datait[1]
        neck = datait[2]
        head = datait[3]
        shoulder_left = datait[4]
        elbow_left = datait[5]
        wrist_left = datait[6]
        hand_left = datait[7]
        shoulder_right = datait[8]
        elbow_right = datait[9]
        wrist_right = datait[10]
        hand_right = datait[11]
        hip_left = datait[12]
        knee_left = datait[13]
        ankle_left = datait[14]
        foot_left = datait[15]
        hip_right = datait[16]
        knee_right = datait[17]
        ankle_right = datait[18]
        foot_right = datait[19]
        spine_shoulder = datait[20]
        handtip_left = datait[21]
        thumb_left = datait[22]
        handtip_right = datait[23]
        thumb_right = datait[24]

        if type=='human':
            data=np.stack((head, neck, spine_shoulder, shoulder_left, shoulder_right, elbow_left, elbow_right,
                                                           wrist_left, wrist_right, thumb_left, thumb_right, hand_left, hand_right, handtip_left,
                                                           handtip_right, spine_mid, spine_base, hip_left, hip_right, knee_left, knee_right,
                                                           ankle_left, ankle_right, foot_left, foot_right))
        else :
            data=np.stack((foot_left, ankle_left, knee_left, hip_left, spine_base, handtip_left, thumb_left,
                                    hand_left, wrist_left, elbow_left, shoulder_left
                                                               ,spine_shoulder,head,neck, shoulder_right,elbow_right,
                                                                    wrist_right, hand_right,thumb_right
                                                                   , handtip_right, spine_mid, hip_right,
                                                                   knee_right, ankle_right,foot_right))
    return data

def transform_image_ludl(image):
    image = np.asarray(image)
    RGB = image
    height = image.shape[1]
    width = image.shape[0]
    X = np.arange(height)
    Y = np.arange(width)
    RGB = np.squeeze(RGB)
    for i in range(3):
        RGB[:, :, i] = np.floor(
            255 * (RGB[:, :, i] - np.amin(RGB[:, :, i])) / (np.amax(RGB[:, :, i]) - np.amin(RGB[:, :, i])))
    img = np.zeros((height, width, 3), dtype=np.uint8)
    for i in X:
        for j in Y:
            img[i, j] = RGB[j, i]
    return img

In [ ]:
def preparation_donnees(lettre,numero):
    seed(1)
    val_ratio = 0.10
    src_directory = '/content/ntu-rgb-120/'
    dest_directory = '/content/ntu_encoder/'
    a = 0
    k=numero
    liste = listdir(src_directory)
    while k < numero + 14310:
        frames = Queue()
        file = liste[k]
        print(lettre + str(k))
        k+=1
        f = file.split('.')
        fichier = open(src_directory+file)
        lignes = fichier.readlines()
        fichier.close()
        nb_positions = int(lignes[0])
        i_lig=1
        while i_lig < len(lignes):
            if "25\n" in lignes[i_lig]:
                i_lig+=1
                i_data = i_lig
                current_frame = []
                while i_lig < i_data + 25:
                    coord = lignes[i_lig].split(" ")
                    current_joint = []
                    X = float(coord[0])
                    Y = float(coord[1])
                    Z = float(coord[2])
                    current_joint.append(X)
                    current_joint.append(Y)
                    current_joint.append(Z)
                    current_frame.append(np.array(current_joint))
                    i_lig+=1
                if(frames.qsize() == max_frames):
                    frames.get()
                    frames.put(np.array(current_frame))
                    img = to_nassim(list(frames.queue),'foot_to_foot')
                    time.sleep(0.01)
                    dossier_destination = dest_directory + "train/" + classes[id_classes.index(f[0][-4:])]
                    if random() < val_ratio:
                        dossier_destination = dest_directory + "test/" + classes[id_classes.index(f[0][-4:])]
                    cv2.imwrite(dossier_destination + "/" + lettre + str(a) + ".png",img)
                    a+=1
                else:
                    frames.put(np.array(current_frame))
            else:
                i_lig+=1

In [ ]:
class Ranger(Thread):
    def __init__(self,lettre,numero):
        Thread.__init__(self)
        self.lettre = lettre
        self.numero = numero
        
    def run(self):
        preparation_donnees(self.lettre, self.numero)

In [10]:
thread = []
thread_count = 8
for i in range(thread_count):
    if i==0:
        lettre="a"
    elif i==1:
        lettre="b"
    elif i==2:
        lettre="c"
    elif i==3:
        lettre="d"
    elif i==4:
        lettre="e"
    elif i==5:
        lettre="f"
    elif i==6:
        lettre="g"
    elif i==7:
        lettre="h"
    downloader = Ranger(lettre,i*14310)
    downloader.start()
    thread.append(downloader)
   
for t in thread:
  t.join()

print("Début zip")
import os
def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths        
  
def nouveau_zip():
    # path to folder which needs to be zipped
    directory = '/content/ntu_encoder/'
  
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)
  
    # writing files to a zipfile
    with zipfile.ZipFile('/content/ntu_encoder_foot_to_foot.zip','w') as zip:
        # writing each file one by one
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
    !cp "/content/ntu_encoder_foot_to_foot.zip" "/content/drive/MyDrive/"
  
print("Fin zip")
nouveau_zip()

Streaming output truncated to the last 5000 lines.
a13670
d56575
h113831
h113832
d56576
b28040
c42370
a13671
f85054
g99585
c42371
f85055
h113833
h113834
h113835
e71037
d56577
h113836
e71038
d56578
b28041
d56579
c42372
h113837
d56580
a13672
a13673
c42373
e71039
g99586
f85056
c42374
e71040
d56581
b28042
g99587
g99588
h113838
f85057
b28043
e71041
b28044
a13674
h113839
h113840
e71042
g99589
g99590
c42375
f85058
e71043
d56582
e71044
h113841
a13675
a13676
b28045
g99591
c42376
c42377
c42378
c42379
d56583
c42380
a13677
d56584
f85059
f85060
h113842
c42381
h113843
e71045
g99592
f85061
c42382
c42383
d56585
h113844
g99593
c42384
b28046
e71046
g99594
a13678
h113845
h113846
c42385
d56586
e71047
d56587
a13679
b28047
b28048
f85062
g99595
f85063
a13680
c42386
b28049
b28050
g99596
g99597
g99598
e71048
h113847
b28051
a13681
c42387
e71049
d56588
b28052
a13682
b28053
b28054
f85064
f85065
h113848
c42388
h113849
f85066
c42389
f85067
e71050
b28055
h113850
h113851
d56589
c42390
e71051
e71052
a13683
f85068
h113

KeyboardInterrupt: ignored

In [ ]:
print("Début zip")
import os
def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths        
  
def nouveau_zip():
    # path to folder which needs to be zipped
    directory = '/content/ntu_encoder/'
  
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)
  
    # writing files to a zipfile
    with zipfile.ZipFile('/content/ntu_encoder_foot_to_foot.zip','w') as zip:
        # writing each file one by one
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
    !cp "/content/ntu_encoder_foot_to_foot.zip" "/content/drive/MyDrive/"
  
print("Fin zip")
nouveau_zip()

Début zip
Fin zip
All files zipped successfully!
